# add_book.ipynb
This notebook guides in the action of adding books to your database.

The notebook is divided in two parts:
1. In the first part we create a minimal database with a single book. We save it as `database.db` in the current folder.
2. In the second part we load the database `database.db` and we add the book contained in the file `newbook.xlsx` (created using `mbp.create_excel_template`).

In [1]:
import polars as pl

import monitorbookprices as mbp

## Create minimal database

In [2]:
book = mbp.new_book(
    {
        'isbn': '9783866473256',
        'author': 'Karl Marx',
        'title': 'Das Kapital',
        'year': '2009',
        'publisher': 'Anaconda',
        'full_price': 7.95,
        'buecher': 'https://www.buecher.de/artikel/buch/das-kapital/25646129/',
        'osiander': 'https://www.osiander.de/shop/home/artikeldetails/A1006759980',
    }
)
df = pl.DataFrame(book, schema_overrides=mbp.schema())
df

isbn,author,title,year,publisher,full_price,adelphi,buecher,feltrinelli,ibs,libraccio,mondadori,osiander
str,str,str,str,str,f64,str,str,str,str,str,str,str
"""9783866473256""","""Karl Marx""","""Das Kapital""","""2009""","""Anaconda""",7.95,null,"""https://www.buecher.de/artikel…",null,null,null,null,"""https://www.osiander.de/shop/h…"


In [3]:
mbp.write_database(
    df,  # dataframe
    'books',  # table name
    url='sqlite:///database.db',  # database url
    if_table_exists='replace',  # behavior if table already exists
)

## Add new book to database

We load the new book from the file `newbook.xlsx`:

In [4]:
df_2 = mbp.read_excel('newbook.xlsx', schema=mbp.schema())
df_2

isbn,author,title,year,publisher,full_price,adelphi,buecher,feltrinelli,ibs,libraccio,mondadori,osiander
str,str,str,str,str,f64,str,str,str,str,str,str,str
"""9781857988826""","""Ursula K. Le Guin""","""The Dispossessed""","""1999""","""Orion Publishing Co""",13.0,null,"""https://www.buecher.de/artikel…","""https://www.lafeltrinelli.it/d…","""https://www.ibs.it/dispossesse…",null,null,"""https://www.osiander.de/shop/h…"


Then, we simply write it to the database specifying the attribute `'append'`:

In [5]:
mbp.write_database(
    df_2,
    table_name='books',
    url='sqlite:///database.db',
    if_table_exists='append',
)

> Note: if we wanted to make sure to not add duplicates (with respect to the ISBN), we can specify a `modify_df_before_writing` function as an argument to `write_database`. For example, we can use `mbp.delete_known_books`, or the following function:
> ```python
> def modify_df(**kwargs):
>     df_db = pl.read_database(
>         query=f'SELECT * FROM {kwargs['table_name']}',
>         connection=kwargs['engine'],
>         )
>     return df_2.join(
>         df_db,
>         on="isbn",
>         how="anti"
>     )
> ```

We check now what has been saved into the database:

In [6]:
df_test = mbp.read_database(
    table_name='books',
    url='sqlite:///database.db',
    schema_overrides=mbp.schema(),
)
df_test

isbn,author,title,year,publisher,full_price,adelphi,buecher,feltrinelli,ibs,libraccio,mondadori,osiander
str,str,str,str,str,f64,str,str,str,str,str,str,str
"""9783866473256""","""Karl Marx""","""Das Kapital""","""2009""","""Anaconda""",7.95,null,"""https://www.buecher.de/artikel…",null,null,null,null,"""https://www.osiander.de/shop/h…"
"""9781857988826""","""Ursula K. Le Guin""","""The Dispossessed""","""1999""","""Orion Publishing Co""",13.0,null,"""https://www.buecher.de/artikel…","""https://www.lafeltrinelli.it/d…","""https://www.ibs.it/dispossesse…",null,null,"""https://www.osiander.de/shop/h…"
